In [1]:
# This notebook will redo the correlation analysis between mRNA and protein data
# across all the 9 tested chemostat experiments
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats import multitest

C:\Users\bioex\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Read in dataset for protein and mRNA seperately
mRNA_data = pd.read_excel("pvsm_new.xlsx",sheet_name='mRNA',index_col=0)
protein_data = pd.read_excel("pvsm_new.xlsx",sheet_name="Protome",index_col=0)
protein_data = protein_data.iloc[:,0:9]
tmp_colnames = protein_data.columns
protein_data.columns = ['D'+name[2:] if 'C_' in name else name for name in tmp_colnames]
mRNA_data_T = mRNA_data.T
protein_data_T = protein_data.T

In [51]:
#Form the spearman correlation dataframe by using scipy.stats.spearman function
m_names=mRNA_data_T.columns
p_names=protein_data_T.columns
names=[]
corrcoefs=[]
pvalues=[]
for name in p_names:
    if name in m_names:
        mRNA = mRNA_data_T[name]
        protein = protein_data_T[name]        
        available_protein_num = sum(~protein.isna())
        if available_protein_num<3 or name in ['YPR124W','YGR027C']:
            continue
        corr,p=stats.spearmanr(mRNA,protein,nan_policy='omit')
        names.append(name)
        corrcoefs.append(corr)
        pvalues.append(p)
pvsm_correlation_df = pd.DataFrame({'Name':names,'coef':corrcoefs,'pvalue':pvalues})

In [52]:
#Get the FDR corrected p-values with a critical FDR = 0.05
res=multitest.multipletests(pvsm_correlation_df.pvalue,method='fdr_tsbky')
corrected_pvsm_correlation_df=pvsm_correlation_df.iloc[res[0],:]
pvsm_correlation_df['pcorr']=res[1]
pvsm_correlation_df['FDR<0.05']=res[0]
#form a datestamp
from datetime import date
today = date.today()
today_str=today.strftime('%Y%m%d')
#pvsm_correlation_df.to_csv('corrected_pvsm_correlation_new'+today_str+'.csv')

In [80]:
# Topy testing
# construct a toy dataframe
name = ['a','b','c','d','e']
coef = [1,2,4,-2,-1]
fdr = [True,False,True,True,False]
toy_dataframe = pd.DataFrame([name,coef,fdr]).T
toy_dataframe.columns=['name','coef','fdr']
toy_dataframe.loc[(~toy_dataframe.fdr)&(toy_dataframe.coef>0),'fdr']=True
toy_dataframe

,name,coef,fdr
0,a,1,True
1,b,2,True
2,c,4,True
3,d,-2,True
4,e,-1,False


In [94]:
# A trying
#print('total rows:',pvsm_correlation_df.shape)
#new_pvsm_correlation_df = pvsm_correlation_df.copy()
#new_pvsm_correlation_df.loc[(pvsm_correlation_df['FDR<0.05'])&(pvsm_correlation_df.coef<=0),'FDR<0.05']=False
#new_pvsm_correlation_df.loc[(pvsm_correlation_df['FDR<0.05'])&(pvsm_correlation_df.coef>0),'FDR<0.05']

total rows: (2568, 5)


1       True
3       True
4       True
5       True
6       True
        ... 
2549    True
2550    True
2558    True
2561    True
2562    True
Name: FDR<0.05, Length: 932, dtype: bool

In [99]:
# hist plot of the correlation coefficient distribution between protein/mRNA pair
import matplotlib.pyplot as plt
import seaborn as sns

#Do the hist plot
# Modify plot by selecting positive significant correlation out, 20211114
new_pvsm_correlation_df = pvsm_correlation_df.copy()
new_pvsm_correlation_df.loc[(pvsm_correlation_df['FDR<0.05'])&(pvsm_correlation_df.coef<=0),'FDR<0.05']=False
ax=sns.histplot(new_pvsm_correlation_df,x="coef",hue="FDR<0.05",kde=True)#,color='black',hist_kws={'linewidth':2,'edgecolor':'black','facecolor':'white'})

#Setting filled density curves
for l in ax.lines:
    x1 = l.get_xydata()[:,0]
    y1 = l.get_xydata()[:,1]
    ax.fill_between(x1,y1,color=l.get_color(),alpha=0.4)
#Set x axis limit
ax.set_xlim(-1,1.1)
#Prepare for annotation in figure
ratio_accepted = sum(new_pvsm_correlation_df["FDR<0.05"])/len(new_pvsm_correlation_df["FDR<0.05"])
ratio_rejected = 1 - ratio_accepted
str_accepted = 'Area ratio: %0.2f'%ratio_accepted
str_rejected = 'Area ratio: %0.2f'%ratio_rejected
#Do annotation for the two density distribution curves
ax.annotate(str_accepted,(0.9,200),(-150,20),textcoords='offset points',color='red',fontsize=12,arrowprops=dict(color='black'))
ax.annotate(str_rejected,(0.6,90),(-150,30),textcoords='offset points',color='blue',fontsize=12,arrowprops=dict(color='black'))

#Formatting the figure
ax.set_xlabel('Spearman correlation coefficient',fontsize=12)
ax.set_ylabel('Count',fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.xaxis.set_tick_params(width=2)
ax.yaxis.set_tick_params(width=2)
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
ax.spines.bottom.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.plot(1.01,0,">k",transform=ax.get_yaxis_transform(),clip_on=False)
ax.plot(-1,1,"^k",transform=ax.get_xaxis_transform(),clip_on=False)
ax.figure.tight_layout()
#plt.show()
plt.savefig('Histplot_of_protein_mRNA_correlation'+today_str+'.png',dpi=300,format='png')